In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv(r'C:\Users\fulian\Desktop\price - 副本.csv')
df_test=pd.read_csv(r"C:\Users\fulian\Desktop\test.csv")

In [3]:
from datetime import datetime
df_train['licenseDate']= pd.to_datetime(df_train["licenseDate"],format='%m/%d/%Y')
df_train['registerDate']= pd.to_datetime(df_train["registerDate"],format='%m/%d/%Y')
df_train['tradeTime']= pd.to_datetime(df_train["tradeTime"],format='%m/%d/%Y')
df_train['duringdate']=df_train['licenseDate']-df_train['registerDate']
df_train['duringdate'] =df_train['duringdate'].astype('timedelta64[D]').astype(float)
dt1=datetime(1900,1,1)
df_train['tradeTime']=df_train['tradeTime']-dt1
df_train['tradeTime'] =df_train['tradeTime'].astype('timedelta64[D]').astype(float)


In [4]:
from datetime import datetime
df_test['licenseDate']= pd.to_datetime(df_test["licenseDate"],format='%m/%d/%Y')
df_test['registerDate']= pd.to_datetime(df_test["registerDate"],format='%m/%d/%Y')
df_test['tradeTime']= pd.to_datetime(df_test["tradeTime"],format='%m/%d/%Y')
df_test['duringdate']=df_test['licenseDate']-df_test['registerDate']
df_test['duringdate'] =df_test['duringdate'].astype('timedelta64[D]').astype(float)
df_test['tradeTime']=df_test['tradeTime']-dt1
df_test['tradeTime'] =df_test['tradeTime'].astype('timedelta64[D]').astype(float)


In [5]:
df_train.dropna(axis=0, subset=['price'], inplace=True)
y = df_train.price
df_train.drop(['price'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype in["object",'datetime64[ns]']]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = df_train[my_cols].copy()

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
from sklearn import tree


# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model= ExtraTreesRegressor()

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
#preds = clf.predict(X_valid)

#print('MAE:', mean_absolute_error(y_valid, preds))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['carid', 'tradeTime',
                                                   'brand', 'serial', 'model',
                                                   'mileage', 'color', 'cityId',
                                                   'carCode', 'transferCount',
                                                   'seatings', 'country',
                                                   'maketype', 'modelyear',
                                                   'displacement', 'gearbox',
                                                   'oiltype', 'newprice',
                                                   'duringdate']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
          

In [ ]:
from sklearn.model_selection  import RandomizedSearchCV
#sorted(model.get_params().keys())
param_dist = {
        'model__n_estimators':range(50,200,1),
        'model__max_depth':range(2,15,1),
        'model__max_features': range(2,8,1)
        }
grid_search = RandomizedSearchCV(clf, param_dist, cv=5,
                           scoring='neg_mean_squared_error',n_iter=300,n_jobs = -1)

grid_search.fit(X_train, y_train)

In [ ]:
df_train.columns


Index(['carid', 'tradeTime', 'brand', 'serial', 'model', 'mileage', 'color',
       'cityId', 'carCode', 'transferCount', 'seatings', 'registerDate',
       'licenseDate', 'country', 'maketype', 'modelyear', 'displacement',
       'gearbox', 'oiltype', 'newprice', 'duringdate'],
      dtype='object')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
# Function for comparing different approaches
def score_dataset1(X_train, X_valid, y_train, y_valid):
    model2=grid_search.best_estimator_
    clf2=Pipeline(steps=[('preprocessor', preprocessor),
                     ('model2', model)
                     ])
    clf2.fit(X_train, y_train)
    preds = model2.predict(X_valid)
    ape=(preds-y_valid)/y_valid
    n=len(y_valid)
    accuracy=len(ape<0.05)/n
    mape=sum(ape)/n
    return accuracy

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model2=grid_search.best_estimator_
    clf2=Pipeline(steps=[('preprocessor', preprocessor),
                     ('model2', model)
                     ])
    clf2.fit(X_train, y_train)
    preds = model2.predict(X_valid)
    ape=(preds-y_valid)/y_valid
    n=len(y_valid)
    mape=sum(ape)/n
    return mape
    

In [ ]:
def measure(mape,accuracy):
    return(0.2*(1-mape)+0.8*accuracy)

In [ ]:
model2=grid_search.best_estimator_
clf2=Pipeline(steps=[('preprocessor', preprocessor),
                     ('model2', model)
                     ])
clf2.fit(X_train, y_train)
preds = clf2.predict(X_valid)
map=score_dataset(X_train, X_valid, y_train, y_valid)
acc=score_dataset1(X_train, X_valid, y_train, y_valid)
print('误差标准:', measure(map,acc))


误差标准: 0.9638718387479623


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5)
scores = cross_val_score(clf2, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=cv)
tree_rmse_scores = np.sqrt(-scores)


In [ ]:
def display_scores(scores):
        print("Scores:", scores)
        print("Mean:", scores.mean())
        print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)

Scores: [ 3.84594123 12.87731439  6.57474143  4.3967353   3.3733324 ]
Mean: 6.213612949720288
Standard deviation: 3.507404680743856


In [ ]:
grid_search.best_params_

{'model__n_estimators': 133, 'model__max_features': 7, 'model__max_depth': 14}

In [ ]:
del df_test['licenseDate']
del df_test['registerDate']


In [ ]:
preds_test=clf2.predict(df_test)


In [ ]:
model.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [ ]:
output = pd.DataFrame({'carid': df_test.carid,
'price': preds_test})
output.to_csv('submission1.csv', index=False)